In [0]:
from pyspark.sql.functions import current_timestamp
from delta.tables import DeltaTable

In [0]:
def add_ingestion_date(input_df):
    return input_df.withColumn("ingestion_timestamp", current_timestamp())

In [0]:
# def save_table(df, db_name, table_name, partition_column):
#     column_list = []
#     for column_name in df.schema.names:
#         if column_name != partition_column:
#             column_list.append(column_name)
#     column_list.append(partition_column)

#     new_df = df.select(column_list)
#     spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic") #only replace partitions with new data received

#     if (spark._jsparkSession.catalog().tableExists(f"{db_name}.{table_name}")):
#         new_df.write.mode("overwrite").insertInto(f"{db_name}.{table_name}")
#     else:
#         new_df.write.mode("overwrite").partitionBy(partition_column).format("parquet").saveAsTable(f"{db_name}.{table_name}")

In [0]:
spark.conf.set("spark.databricks.optimizer.dynamicPartitionPruning", "true")

def merge_table(df, db_name, table_name, partition_column, merge_condition):
    column_list = []
    for column_name in df.schema.names:
        if column_name != partition_column:
            column_list.append(column_name)
    column_list.append(partition_column)

    new_df = df.select(column_list)

    folder_name = db_name[3:]

    if (spark._jsparkSession.catalog().tableExists(f"{db_name}.{table_name}")):
        deltaTable = DeltaTable.forPath( spark, f"/mnt/formula1jf/{folder_name}/{table_name}")
        deltaTable.alias("tgt").merge(
            new_df.alias("src"),
            merge_condition) \
            .whenMatchedUpdateAll() \
            .whenNotMatchedInsertAll() \
            .execute()
    else:
        new_df.write.mode("overwrite").partitionBy(partition_column).format("delta").saveAsTable(f"{db_name}.{table_name}")

In [0]:
name = "f1_processed"
print(name[3:])

processed
